In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from tqdm.notebook import tqdm
from GalPop import GalPop

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.27)


In [10]:
# Compile populations
sig_cube = fits.open(r"C:\Users\sikor\OneDrive\Desktop\benpy\FinalTry\Hyperion_interpolated_cube_sigma.fits")
sigs = np.array([2.0, 2.25] + list(np.arange(2.5,5.5,0.5)))
pk_path = r"C:\Users\sikor\OneDrive\Desktop\BigData\Hyperion\HyperionPeaks\\"
pk_sums = [f"peaks_sig{s}.dat" for s in sigs]

gps = [GalPop.loadFile(f'Final_GPs/MC_{n}.npy') for n in range(0,101)]

In [13]:
# Interpolated sigma cube
sig_cube = fits.open(r"C:\Users\sikor\OneDrive\Desktop\benpy\ThirdTry\Hyperion_interpolated_cube_sigma.fits")

# Path for folders with .dat sigma files
sigPath = r"C:\Users\sikor\OneDrive\Desktop\benpy\HyperionPeaks"

# Define sigma information
sigs = [2.0]
sig_path = sigPath

sig_sums = [sigPath + f"\peaks_sig{s}.dat" for s in sigs]
sig_folders = [sigPath+ f"\sig{s}_peaks" for s in sigs]

# Assign peaks to objects
for _id, gp in tqdm(enumerate(gps), total=len(gps)):
    gp.assignPeaks(sigs, sig_cube, sig_sums, sig_folders, overwrite=True)
    gp.saveFile(f"Final_GPs/MC_{_id}")

  0%|          | 0/101 [00:00<?, ?it/s]

In [17]:
gp.pks.keys()

dict_keys([2.0])

In [15]:
# Make SubPops

for g in tqdm(gps):
    
    g.subPop("field1", [2.0, 4.0], 13. , [2.15, 2.25], pk_path, pk_sums[0], sig_cube, pk_folder='sig2.0_peaks', cosmo = cosmo)
    g.subPop("field2", [2.0, 4.0], 13. , [2.8, 2.9], pk_path, pk_sums[0], sig_cube, pk_folder='sig2.0_peaks', cosmo = cosmo)


    for s, ps in zip(sigs[1:], pk_sums[1:]):

        g.subPop(s, [s, np.inf], 13.0, [2.2, 3.], pk_path, ps, sig_cube, pk_nums=[1])

    




  0%|          | 0/101 [00:00<?, ?it/s]

self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assignPeaks with this key
self.pks has no key 4.0. Run self.assign

In [ ]:
### BUILD SMFs ###
sp_keys = [['field1', 'field2']] + ['o2.5', 'o3.0', 'o4.0']
smf_keys = ['field', 'o2.5', 'o3.0', 'o4.0']

for idx, t in enumerate(gps):
    t.subPop()

    # Make SMFs #    
    m_range = [9.5, 11.75, 0.25]
    t.make_SMF(sp_keys, smf_keys ,m_range)
    t.make_SMF(['o4.0', ['field1', 'field2']], ['o4.0', 'f2'] ,[9.5, 11.75, 1/3])




    # Make fits #
    # t.fit_SMF(smf_keys, smf_keys, [1 for s in smf_keys], p0=[10, 1, -1], maxfev=3000)
    
    # t.SMF_plot(smf_keys, smf_keys, smf_keys, smf_keys, title="", plot="None")

sp_keys += ['23', '34']
smf_keys += ['23', '34']

for idx, h in enumerate(h_pops):
    # Make SMFs #
    m_range = [9.5, 11.75, 0.25]
    h.make_SMF(sp_keys[1:], smf_keys[1:] ,m_range)
    h.make_SMF(['o4.0'], ['o4.0'] ,[9.5, 11.75, 1/3])
